In [1]:
import pandas as pd
from pyspark.sql import functions as fn

In [2]:
# lets define a simple dataframe

data = {'A': ['a', 'b', None], 
        'B': [0, 1, 0],
        'C': [1, 0, 1],
        'D': [1, 0, 0]}
df = pd.DataFrame(data)
df = spark.createDataFrame(df)
display(df)

A,B,C,D
a,0,1,1
b,1,0,0
null,0,1,0


In [3]:
# now lets filter with a condition and its inverse - watch what happens to the row with a null value
df.show()
df.filter(df.A == "a").show()
df.filter(df.A != "a").show()

+----+---+---+---+
 A| B| C| D|
+----+---+---+---+
 a| 0| 1| 1|
 b| 1| 0| 0|
null| 0| 1| 0|
+----+---+---+---+

+---+---+---+---+
 A| B| C| D|
+---+---+---+---+
 a| 0| 1| 1|
+---+---+---+---+

+---+---+---+---+
 A| B| C| D|
+---+---+---+---+
 b| 1| 0| 0|
+---+---+---+---+

In [4]:
# because of pysparks SQL compatibility column names are not case sensitive
df2 = df.withColumnRenamed("B", "a")
df2.show()
# df2.select("A").show()
df2.select("a").show()
df2.withColumnRenamed("A", "e").show()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-929035903441204> in <module> () 
 2 df2 . show ( ) 
 3 # df2.select("A").show() 
 ----> 4 df2 . select ( "a" ) . show ( ) 
 5 df2 . withColumnRenamed ( "A" , "e" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1343 [ Row ( name = u'Alice' , age = 12 ) , Row ( name = u'Bob' , age = 15 ) ] 
 1344 """
 -> 1345 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1346 return DataFrame ( jdf , self . sql_ctx ) 
 1347 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : "Reference 'a' is ambiguous, could be: a, a.;"

+----+
 a|
+----+
 a|
 b|
null|
+----+

In [6]:
# sufficiently large ints will roll over to null without raising an error
# also floats seem to be able to hold larger abosolute values than they can operate on

data = {'LongIntCol': [1234567890, 123456789012345678, 1, 12345678901234567890, 1],
        'FloatCol': [1.0, 1.0, 123456789012345678, 1.0, 12345678901234567890.0]
       }
df = pd.DataFrame(data)
df = spark.createDataFrame(df)
df = df.withColumn("Multiplied", df.LongIntCol*df.FloatCol)
display(df)

FloatCol,LongIntCol,Multiplied
1.0,1234567890,1.23456789E9
1.0,123456789012345678,1.2345678901234568E17
1.2345678901234568E17,1,1.2345678901234568E17
1.0,null,null
1.2345678901234567E19,1,1.2345678901234567E19
